# Testing MCTS with Various Models
Here we will test MCTS first with the DKT model

In [35]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from concept_dependency_graph import ConceptDependencyGraph
from simple_mdp import create_custom_dependency
import data_generator as dg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
n_concepts = 5
concept_tree = create_custom_dependency()
print ("Initializing synthetic data sets...")
n_students = 100000
seqlen = 10
policy = 'random'
filename = "test.pickle".format(n_students, seqlen, policy)
dg.generate_data(concept_tree, n_students=n_students, seqlen=seqlen, policy=policy, filename="{}{}".format(dg.SYN_DATA_DIR, filename))
print ("Data generation completed. ")

Initializing synthetic data sets...
Generating data for 100000 students with behavior policy random and sequence length 10.
Data generation completed. 


In [65]:
import dynamics_model_class as dmc
import numpy as np
import dataset_utils

In [76]:
dmodel = dmc.DynamicsModel(model_id="test_model", timesteps=seqlen, load_checkpoint=False)

Loading RNN dynamics model...
Directory path for tensorboard summaries: ../tensorboard_logs/test_model/
Checkpoint directory path: ../checkpoints/test_model/
Model loaded.


In [77]:
# load toy data
data = dataset_utils.load_data(filename="../synthetic_data/test.pickle")
input_data_, output_mask_, target_data_ = dataset_utils.preprocess_data_for_rnn(data)

In [78]:
train_data = (input_data_[:,:,:], output_mask_[:,:,:], target_data_[:,:,:])

In [79]:
dmodel.train(train_data, n_epoch=10, load_checkpoint=True)

Training Step: 10060  | total loss: 0.01693 | time: 3.735s
| Adam | epoch: 008 | loss: 0.01693 -- iter: 13504/90000


KeyboardInterrupt: 

In [80]:
preds = dmodel.predict(input_data_[:1,:2, :])
print len(preds)
print len(preds[0])
print preds[0]

1
10
[[0.7306510806083679, 0.7202691435813904, 0.4735923707485199, 0.5003368258476257, 0.024046525359153748], [0.8806740641593933, 0.8737850189208984, 0.49894294142723083, 0.5077625513076782, 0.009034689515829086], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0]]


In [81]:
rnnsim = dmc.RnnStudentSim(dmodel)

In [82]:
from mcts import *
actix = 0
actvec = np.zeros((n_concepts,))
actvec[actix] = 1
act = StudentAction(actix, actvec)
print rnnsim.sequence
print rnnsim.sample_observations()

[]
None


In [83]:
rnnsim.advance_simulator(act, 1)


In [84]:
print rnnsim.sequence
print rnnsim.sample_observations()
print rnnsim.sample_reward()

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]
[0.7307887673377991, 0.4983048737049103, 0.5103313326835632, 0.030637890100479126, 0.0197952501475811]
1.78985811397
